## Needed libraries

In [4]:
%matplotlib inline
import cv2 # OpenCV for perspective transform
import numpy as np
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import scipy.misc # For saving images as needed
import glob  # For reading in a list of images from a folder
#imageio.plugins.ffmpeg.download()

## In the decision.py file 

1. This function is called when the rover is stuck in an obstacle in order to solve this problem.

In [2]:
def stuck(Rover):
    global stuck_time
    Rover.throttle = 0
    Rover.brake = 0
    Rover.steer = -15
    return Rover

2. We defined decision step in where we made use of stuck_time - which is the time taken by the rover being stuck in an obstacle or donut_time - which is time taken by the rover revolving around itself in a circle.

Lastly, the offset where it makes the rover stay to the left side of the navigable terrain (left wall hugging)
    to get the right wall --> multiply the offset by -1


In [3]:

def decision_step(Rover):   
    global stuck_time
    global donut_time
    
    offset = 0

    # Only apply left wall hugging when out of the starting point (after 10s)
    # to avoid getting stuck in a circle
    if Rover.total_time > 10:
        # Steering proportional to the deviation results in
        # small offsets on straight lines and
        # large values in turns and open areas
        offset = 0.8 * np.std(Rover.nav_angles)

2. 1. This if condition is responsible on sending pickup if a rock was detected.  

In [10]:
if Rover.near_sample and Rover.vel == 0 and not Rover.picking_up:
        Rover.send_pickup = True
        return Rover

SyntaxError: 'return' outside function (1536536892.py, line 3)

2.2 This if condition checks whether we are near a sample or not, in case true we change the rover mode to stop

In [6]:
if Rover.near_sample:
        Rover.mode = 'stop'
        Rover.throttle = 0
        Rover.brake = Rover.brake_set
        return Rover

SyntaxError: 'return' outside function (2926484530.py, line 5)

2.3 This if condition is responsible on checking whether the rover mode is in LargeCircleRotation, in case true the rover moves forward 

In [ ]:
if Rover.mode == "LargeCircleRotation":
        if time.time() - donut_time > 15:
            Rover.mode = "forward"
            donut_time=time.time()
        else:
            Rover.throttle = 0
            Rover.brake = 0
            Rover.steer = -10
        return Rover

2.4 This if condition is responsible on checking whether the rover rotating in the same direction, in case true change rover mode to LargeCircleRotation 

In [ ]:
if Rover.steer==15 or Rover.steer==-15:
        if time.time() - donut_time > 10:  
            Rover.mode = "LargeCircleRotation"
            return Rover
else:
        donut_time=time.time()

2.5 This if condtion checks whether the rover can see the rock in its feild of view, in case true it stops immediately and moves forward towards the rock. The same code is repeated in the stop mode of the rover.

In [ ]:
if len(Rover.rock_angles) > 1:
            Rover.brake = Rover.brake_set
            Rover.mode = 'stop'
            Rover.steer = np.clip(np.mean(Rover.rock_angles * 180/np.pi), -15, 15)
            if (np.mean(Rover.rock_dists) >= 4):
                Rover.throttle = 0.4
                Rover.brake = 0
            elif (np.mean(Rover.rock_dists) <= 4):
                Rover.brake = Rover.brake_set
                Rover.near_sample = True
            return Rover

In [ ]:
elif Rover.mode == 'stop':
            # If we're in stop mode but still moving keep braking
            if len(Rover.rock_angles) > 1:
                Rover.brake = Rover.brake_set
                Rover.steer = np.clip(np.mean(Rover.rock_angles * 180/np.pi), -15, 15)
                if (np.mean(Rover.rock_dists) >= 3):
                    Rover.throttle = 0.2
                    Rover.brake = 0
                elif (np.mean(Rover.rock_dists) <= 3):
                    Rover.brake = Rover.brake_set
                    Rover.near_sample = True
                return Rover

# 2.6 In case of forward state: 

2.6.1 check if the rover is stuck in forward mode and not moving (stuck under a rock)

In [ ]:
 if Rover.vel <= 0.1 and Rover.throttle!=0 and len(Rover.rock_angles == 0):
                if time.time() - stuck_time > 1.5:
                    Rover = stuck(Rover)
                    return Rover

2.6.2 check for the navigable terrain and apply a throttle in the direction of the mean of the navigation angles

In [7]:
if len(Rover.nav_angles) >= Rover.stop_forward:  
                # If mode is forward, navigable terrain looks good 
                # and velocity is below max, then throttle 
                if Rover.vel < Rover.max_vel:
                    # Set throttle value to throttle setting
                    Rover.throttle = Rover.throttle_set
                else: # Else coast
                    Rover.throttle = 0
                Rover.brake = 0
                # Set steering to average angle clipped to the range +/- 15
                try:
                    Rover.steer = np.clip(np.mean(Rover.nav_angles * 180/np.pi) + offset, -15, 15)
                except:
                    print("mean of nav angles caught an exception")

NameError: name 'Rover' is not defined

2.6.3 else in case of a lack of navigable terrain => switch to stop mode where we navigate to the right (-15) in order to find navigable terrain

In [ ]:
elif len(Rover.nav_angles) < Rover.stop_forward:
                    # Set mode to "stop" and hit the brakes!
                    Rover.throttle = 0
                    # Set brake to stored brake value
                    Rover.brake = Rover.brake_set
                    Rover.steer = np.clip(np.mean(Rover.nav_angles * 180/np.pi)+ offset, -15, 15)
                    Rover.mode = 'stop'

2.7 as mentioned above in the stop mode:

2.7.1 we check whether there are rock samples or not, in case true we break then steer towards the samples

In [ ]:
elif Rover.mode == 'stop':
            # If we're in stop mode but still moving keep braking
            if len(Rover.rock_angles) > 1:
                Rover.brake = Rover.brake_set
                Rover.steer = np.clip(np.mean(Rover.rock_angles * 180/np.pi), -15, 15)
                if (np.mean(Rover.rock_dists) >= 3):
                    Rover.throttle = 0.2
                    Rover.brake = 0
                elif (np.mean(Rover.rock_dists) <= 3):
                    Rover.brake = Rover.brake_set
                    Rover.near_sample = True
                return Rover

2.7.2 we check if we did not stop completely then we set the brakes and stop 

In [ ]:
if Rover.vel > 0.2:
                Rover.throttle = 0
                Rover.brake = Rover.brake_set
                Rover.steer = 0

2.7.3 we check if the velocity is below 0.2 and there isnt enough navigable terrain, in case true we steer to the right - since we are hugging the left wall, in case false if there is enough navigable terrain we move in the direction of the mean of the angles of the navigable terrain

In [ ]:
elif Rover.vel <= 0.2:
                # Now we're stopped and we have vision data to see if there's a path forward
                if (len(Rover.nav_angles) < Rover.go_forward) and len(Rover.rock_angles) == 0:
                    Rover.throttle = 0
                    # Release the brake to allow turning
                    Rover.brake = 0
                    # Turn range is +/- 15 degrees, when stopped the next line will induce 4-wheel turning
                    Rover.steer = -15 # Could be more clever here about which way to turn
                # If we're stopped but see sufficient navigable terrain in front then go!
                if len(Rover.nav_angles) >= Rover.go_forward:
                    # Set throttle back to stored value
                    Rover.throttle = Rover.throttle_set
                    # Release the brake
                    Rover.brake = 0
                    # Set steer to mean angle
                    try:
                        Rover.steer = np.clip(np.mean(Rover.nav_angles  * 180/np.pi) + offset, -15, 15)
                    except:
                        print("mean of nav angles caught an exception")
                    Rover.mode = 'forward'

2.8 At last we check if the rover is stuck in a circle then we start moving forward as soon as we see little navigable terrain  

In [ ]:
if Rover.vel == 0 and Rover.steer == -15 and len(Rover.nav_angles) > 20 and len(Rover.rock_angles) == 0 :
                Rover.steer = 0
                Rover.throttle = 0.5
                Rover.mode = 'forward'

## In the perception.py we made some changes

1. we defined impose_range function to limit the range of the rover's camera in order to increase the fidelity 

In [ ]:
def impose_range(xpix, ypix, range=80):
    dist = np.sqrt(xpix**2 + ypix**2)
    return xpix[dist < range], ypix[dist < range]

1.1 Then we apply the limited range only to the obstacles and the navigable terrain

In [ ]:
x_thresh, y_thresh = impose_range(x_thresh,y_thresh)
x_obst , y_obst = impose_range(x_obst , y_obst)

2. added code to update the world map only when the rover is flat on the ground 

In [ ]:
if (Rover.roll < 0.5 or Rover.roll > 359.5) and (Rover.pitch < 0.5 or Rover.pitch > 359.5):
        Rover.worldmap[yw_obst, xw_obst, 0] =255 #+= 1
        Rover.worldmap[yw_rock, xw_rock, 1] =255 #+= 1
        Rover.worldmap[y_world, x_world, 2] =255 #+= 1

        nav_pix = Rover.worldmap[:, :, 2] > 0
        Rover.worldmap[nav_pix, 0] = 0
        # clip to avoid overflow
        Rover.worldmap = np.clip(Rover.worldmap, 0, 255)
        Rover.worldmap[yw_obst , xw_obst, 0] += 1
        Rover.worldmap[yw_rock, xw_rock, 1] += 1
        Rover.worldmap[y_world, x_world, 2] += 1